In [1]:
import os
import librosa
# import pydub
from pydub import AudioSegment
import numpy as np
import copy
import random
from scipy import signal as scipy_signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import datetime

from torch.utils.tensorboard import SummaryWriter

In [2]:
dataset_dir = '/home/torooc/dataHDD2/speech_commands_v0.01/'

In [3]:
testing_list_path = dataset_dir + 'testing_list.txt'
validation_list_path = dataset_dir + 'validation_list.txt'

In [4]:
with open(testing_list_path) as f:
    testing_list = f.readlines()
    testing_list = [file.strip() for file in testing_list]
    
with open(validation_list_path) as f:
    validation_list = f.readlines()
    validation_list = [file.strip() for file in validation_list]

In [5]:
command_dirs = os.listdir(dataset_dir)

command_dirs = [direc for direc in command_dirs if os.path.isdir(os.path.join(dataset_dir, direc))]

command_dirs.sort()

command_dirs.remove('_background_noise_')

In [6]:
print(command_dirs)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [7]:
key2label_dict = dict()
label2key_list = list()

key2label_dict[' '] = 0
label2key_list.append(' ')

for i, command in enumerate(command_dirs):
    key2label_dict[command] = i + 1
    label2key_list.append(command)

In [8]:
key2label_dict

{' ': 0,
 'bed': 1,
 'bird': 2,
 'cat': 3,
 'dog': 4,
 'down': 5,
 'eight': 6,
 'five': 7,
 'four': 8,
 'go': 9,
 'happy': 10,
 'house': 11,
 'left': 12,
 'marvin': 13,
 'nine': 14,
 'no': 15,
 'off': 16,
 'on': 17,
 'one': 18,
 'right': 19,
 'seven': 20,
 'sheila': 21,
 'six': 22,
 'stop': 23,
 'three': 24,
 'tree': 25,
 'two': 26,
 'up': 27,
 'wow': 28,
 'yes': 29,
 'zero': 30}

In [9]:
total_wav_file_list = list()

for command_dir in command_dirs:
    wav_list = os.listdir(os.path.join(dataset_dir, command_dir))
    
    wav_list_with_prefix = [os.path.join(command_dir, file_name) for file_name in wav_list]
    
    total_wav_file_list += wav_list_with_prefix

In [10]:
training_list = list(set(total_wav_file_list) - set(testing_list) - set(validation_list))
training_list.sort()


In [11]:
'''
validation_list
testing_list
training_list
'''

'\nvalidation_list\ntesting_list\ntraining_list\n'

In [12]:
audio_metadata = list()

for wav_file_name in total_wav_file_list:
    audio = AudioSegment.from_wav(dataset_dir + wav_file_name)
    fs = audio.frame_rate
    length_in_samples = audio.frame_count()
    
#   normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
#   print("Max: {}, Min: {}".format(max(normalized_audio_array), min(normalized_audio_array)))
    
    audio_metadata.append([fs, length_in_samples])

In [13]:
SR = 16000

class DatasetLoader():
    
    def __init__(self, batch_size, dataset_dir, wav_file_name_list, key2label, label2key):
        self.batch_size = batch_size
        self.dataset_dir = dataset_dir
        self.wav_file_name_list = copy.deepcopy(wav_file_name_list)
        self.sr = SR
        self.nsc_in_ms = 40
#         self.nov_in_ms = self.nsc_in_ms / 2
        self.nov_in_ms = 0
        self.nsc_in_sample = int(self.nsc_in_ms / 1000 * self.sr)
        self.nov_in_sample = int(self.nov_in_ms / 1000 * self.sr)
        
        self.key2label = key2label
        self.label2key = label2key
        
    def shuffle_dataset_order(self):
        random.shuffle(self.wav_file_name_list)
        
    def batch_generator(self):
        
        self.shuffle_dataset_order()
        
        spectrogram_list = list()
        label_list = list()
        
        for i, wav_file_name in enumerate(self.wav_file_name_list):
            
            audio = AudioSegment.from_wav(self.dataset_dir + wav_file_name)
            
            normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
            f, t, Zxx = scipy_signal.stft(normalized_audio_array, fs=self.sr, 
                                          nperseg=self.nsc_in_sample,
                                          noverlap=self.nov_in_sample)
        
            Sxx = np.abs(Zxx)
            
            normalized_spectrogram = (20 * np.log10(np.maximum(Sxx, 1e-8)) + 160) / 160
            spectrogram_list.append(normalized_spectrogram)
            
            keyword = wav_file_name.split('/')[0]
            label = self.key2label[keyword]
            label_list.append(label)
            

            if (i + 1) % self.batch_size == 0 or i + 1 == len(self.wav_file_name_list):
                
                spectrogram_time_step_list = [specgram.shape[1] for specgram in spectrogram_list]
                
                max_time_step = max(spectrogram_time_step_list)
                
                freq_size = spectrogram_list[0].shape[0]
                
                batch = np.zeros([len(spectrogram_list), freq_size, max_time_step])
                
                for j, specgram in enumerate(spectrogram_list):
                    batch[j, :specgram.shape[0], :specgram.shape[1]] = specgram
                
                spectrogram_list = list()
                
#                 print('{}:{}'.format(i, batch.shape[0]))
            
                yield batch, np.asarray(label_list)
                
                label_list = list()
                
#                 break
    
#             break

In [14]:
len(training_list)

51088

In [15]:
dataset_loader_training = DatasetLoader(256, dataset_dir, training_list, key2label_dict, label2key_list)
dataset_loader_testing = DatasetLoader(256, dataset_dir, testing_list, key2label_dict, label2key_list)
dataset_loader_validation = DatasetLoader(256, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_testing = DatasetLoader(64, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(64, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_training = DatasetLoader(768, dataset_dir, training_list, key2label_dict, label2key_list)
# dataset_loader_testing = DatasetLoader(768, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(768, dataset_dir, validation_list, key2label_dict, label2key_list)

In [16]:
# batch_generator = dataset_loader_training.batch_generator()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# class basic_model(nn.Module):
#     def __init__(self, D_in, H, num_layers, num_labels):
#         super(basic_model, self).__init__()
#         self.fc = torch.nn.Linear(D_in, H)
#         self.relu = torch.nn.ReLU()
#         self.dropout = nn.Dropout(p=0.2)

#         self.gru_layers = nn.ModuleList([nn.GRU(H, int(H / 2), bidirectional=True, batch_first=True) for i in range(num_layers)])

#         self.fc_pred = nn.Linear(H, num_labels)
#         self.log_softmax = nn.LogSoftmax(dim=2)
        
#     def forward(self, input_tensor):
#         # (B, T, F)
#         output_tensor = self.fc(input_tensor)
#         output_tensor = self.relu(output_tensor)
#         output_tensor = self.dropout(output_tensor)
#         # (B, T, H)
#         for layer in self.gru_layers:
#             output_tensor, _ = layer(output_tensor)
            
#         output_tensor = self.fc_pred(output_tensor)

#         output_tensor = self.log_softmax(output_tensor)
        
#         return output_tensor

class basic_conv_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_conv_model, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 16, (13, 2))
        self.conv2d_2 = nn.Conv2d(16, 32, (13, 2))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(4800, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)
        tensor = self.conv2d_1(input_tensor)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        tensor = self.conv2d_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor
    
class basic_crnn_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_crnn_model, self).__init__()
        self.conv2d = nn.Conv2d(1, 16, (13, 2))
        self.gru_1 = nn.GRU(16 * 103, 128, 1)
        self.gru_2 = nn.GRU(128, 128, 1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128 * 12, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)  # (B, 1, F, T)
        tensor = self.conv2d(input_tensor) # (B, 16, F, T)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        
        tensor = tensor.view(tensor.shape[0], -1, tensor.shape[3])
        # (B, 16 * F, T)
        
#         print('[After Reshape]: {}'.format(tensor.shape))
        
        tensor.transpose_(0, 2) # (T, F, B)
        tensor.transpose_(1, 2) # (T, B, F)

        tensor, _ = self.gru_1(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True) # (T, B, F)
#         tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))

        tensor, _ = self.gru_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True) # (T, B, F)

        tensor.transpose_(0, 1) # (B, T, F)
    
#         print('[After Reshape]: {}'.format(tensor.shape))
    
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor

    
# model = basic_model(321, 512, 3, 31).float().to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# dictation_loss = nn.CTCLoss().to(device)

model = basic_crnn_model().float().to(device)  
optimizer = optim.Adam(model.parameters(), lr=1e-4)
        

In [19]:
# batch_generator = dataset_loader_training.batch_generator()
# next(batch_generator)[0].shape

In [20]:
MAX_EPOCH = 300

writer = SummaryWriter(log_dir='runs/crnn_log')

for epoch in range(MAX_EPOCH):
    training_loss_list = list()
    batch_generator = dataset_loader_training.batch_generator()
    model.train()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
        optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
        loss.backward()
        training_loss_list.append(loss.item())
        optimizer.step()
        
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_train = correct_num / total_num

    testing_loss_list = list()

    batch_generator = dataset_loader_testing.batch_generator()

    model.eval()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        testing_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_test = correct_num / total_num

    validation_loss_list = list()

    batch_generator = dataset_loader_validation.batch_generator()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        validation_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_valid = correct_num / total_num
    
    training_loss_mean = np.mean(training_loss_list)
    testing_loss_mean = np.mean(testing_loss_list)
    validation_loss_mean = np.mean(validation_loss_list)
    
    print(datetime.datetime.now().time())
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, training_loss_mean, testing_loss_mean, validation_loss_mean))
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, acc_train, acc_test, acc_valid))
    
    writer.add_scalar('Loss/Train', training_loss_mean, epoch)
    writer.add_scalar('Loss/Test', testing_loss_mean, epoch)
    writer.add_scalar('Loss/Valid', validation_loss_mean, epoch)
    writer.add_scalar('Acc/Train', acc_train, epoch)
    writer.add_scalar('Acc/Test', acc_test, epoch)
    writer.add_scalar('Acc/Valid', acc_valid, epoch)
    

09:56:18.907031
#000 [Train: 3.373] [Test: 3.214] [Valid: 3.195]
#000 [Train: 0.052] [Test: 0.097] [Valid: 0.102]
09:57:01.858595
#001 [Train: 2.995] [Test: 2.857] [Valid: 2.811]
#001 [Train: 0.133] [Test: 0.161] [Valid: 0.170]
09:57:44.780909
#002 [Train: 2.722] [Test: 2.626] [Valid: 2.570]
#002 [Train: 0.194] [Test: 0.230] [Valid: 0.242]
09:58:27.698783
#003 [Train: 2.482] [Test: 2.394] [Valid: 2.339]
#003 [Train: 0.270] [Test: 0.300] [Valid: 0.312]
09:59:10.729282
#004 [Train: 2.279] [Test: 2.215] [Valid: 2.169]
#004 [Train: 0.329] [Test: 0.355] [Valid: 0.365]
09:59:53.807721
#005 [Train: 2.105] [Test: 2.052] [Valid: 2.010]
#005 [Train: 0.383] [Test: 0.405] [Valid: 0.418]
10:00:36.918494
#006 [Train: 1.921] [Test: 1.878] [Valid: 1.827]
#006 [Train: 0.438] [Test: 0.453] [Valid: 0.461]
10:01:20.052360
#007 [Train: 1.742] [Test: 1.714] [Valid: 1.673]
#007 [Train: 0.495] [Test: 0.500] [Valid: 0.521]
10:02:03.083466
#008 [Train: 1.597] [Test: 1.596] [Valid: 1.557]
#008 [Train: 0.537] [Te

10:47:55.435172
#072 [Train: 0.295] [Test: 0.424] [Valid: 0.431]
#072 [Train: 0.913] [Test: 0.872] [Valid: 0.873]
10:48:38.346640
#073 [Train: 0.290] [Test: 0.439] [Valid: 0.435]
#073 [Train: 0.914] [Test: 0.870] [Valid: 0.870]
10:49:21.257843
#074 [Train: 0.286] [Test: 0.428] [Valid: 0.423]
#074 [Train: 0.917] [Test: 0.874] [Valid: 0.875]
10:50:04.306388
#075 [Train: 0.280] [Test: 0.431] [Valid: 0.428]
#075 [Train: 0.918] [Test: 0.874] [Valid: 0.873]
10:50:47.338657
#076 [Train: 0.280] [Test: 0.434] [Valid: 0.435]
#076 [Train: 0.917] [Test: 0.870] [Valid: 0.873]
10:51:30.339826
#077 [Train: 0.274] [Test: 0.419] [Valid: 0.423]
#077 [Train: 0.920] [Test: 0.876] [Valid: 0.877]
10:52:13.315375
#078 [Train: 0.269] [Test: 0.425] [Valid: 0.441]
#078 [Train: 0.921] [Test: 0.873] [Valid: 0.873]
10:52:56.353072
#079 [Train: 0.270] [Test: 0.420] [Valid: 0.425]
#079 [Train: 0.920] [Test: 0.877] [Valid: 0.875]
10:53:39.580271
#080 [Train: 0.266] [Test: 0.411] [Valid: 0.410]
#080 [Train: 0.922] [Te

11:39:13.781862
#144 [Train: 0.129] [Test: 0.386] [Valid: 0.394]
#144 [Train: 0.963] [Test: 0.893] [Valid: 0.896]
11:39:56.415461
#145 [Train: 0.126] [Test: 0.369] [Valid: 0.387]
#145 [Train: 0.965] [Test: 0.897] [Valid: 0.898]
11:40:39.041871
#146 [Train: 0.124] [Test: 0.377] [Valid: 0.384]
#146 [Train: 0.965] [Test: 0.894] [Valid: 0.894]
11:41:21.677739
#147 [Train: 0.123] [Test: 0.376] [Valid: 0.380]
#147 [Train: 0.964] [Test: 0.897] [Valid: 0.897]
11:42:04.275496
#148 [Train: 0.122] [Test: 0.390] [Valid: 0.389]
#148 [Train: 0.966] [Test: 0.895] [Valid: 0.896]
11:42:46.901326
#149 [Train: 0.120] [Test: 0.382] [Valid: 0.391]
#149 [Train: 0.965] [Test: 0.898] [Valid: 0.892]
11:43:29.483853
#150 [Train: 0.118] [Test: 0.375] [Valid: 0.383]
#150 [Train: 0.967] [Test: 0.897] [Valid: 0.895]
11:44:12.085192
#151 [Train: 0.118] [Test: 0.384] [Valid: 0.381]
#151 [Train: 0.966] [Test: 0.898] [Valid: 0.895]
11:44:54.683724
#152 [Train: 0.116] [Test: 0.384] [Valid: 0.401]
#152 [Train: 0.967] [Te

KeyboardInterrupt: 

In [ ]:
# label.shape

In [ ]:
# output = model(torch.cuda.FloatTensor(batch))

In [ ]:
# output.shape

In [ ]:
# torch.tensor(batch).unsqueeze_(1).shape

In [ ]:
# input_tensor = torch.tensor(batch).to(device).float()
# input_tensor = input_tensor.transpose(1, 2)
# model(input_tensor).shape

In [ ]:
# for batch, label_list in batch_generator:
#     print(batch.shape[0])
#     plt.figure(figsize=(5, 1))
#     plt.plot(label_list)
#     plt.show()
    
#     plt.figure(figsize=(6, 18))
# #     plt.imshow(batch[0], aspect='auto')
#     plt.imshow(batch[0])
#     plt.show()
    
#     break